In [2]:
import gym 
import numpy as np
import random
from copy import deepcopy

In [28]:
class CoopEnv(gym.Env):
    
    def characteristic_function(self, coalition, singleton_vals):
        
        """ Return the characteristic value of a coalition
            Input: 'coalition' - Set of players present within a coalition of a coalition structure
                   'singleton_vals' - Dictionary containing singleton values for each player
            Output: 'value' - float representing value of input coalition
        
        """
        
        value = sum(singleton_vals[f'Player {player}'] for player in coalition) * len(coalition)
        return value
    
    def get_sum(self, coalition, singleton_vals):
        
        """ Return the sum of values of individual players in a coalition.
            Input: 'coalition' - Set of players present within a coalition of a coalition structure
                   'singleton_vals' - Dictionary containing singleton values for each player
            Output: 'value' - float representing value of input coalition
        
        """
        
        value = sum(singleton_vals[f'Player {player}'] for player in coalition) 
        return value 
    
    
    def get_observations_from_CS(self, CS, locations, n):
        
        """ Given a coalition structure, return ordered list of binary strings representing set observations
            for each agent (i.e what set the agent is in)
            Input: 'CS' - List of sets representing current coalition structure of the game.
                   'locations' - Dictionary returning index representing occupied coalition index.
                   'n' - Number of players.
            Output: 'binary_list' - List of arrays containing binary string representation of each observed set
                                     as observed by each agent.
        
        """
        
        binary_list = []
        generic_string = np.zeros((n))
        observations = [CS[locations[f'Player {i+1}']] for i in range(n)] # for each agent, get the coalition they occupy
        
        for subset in observations:
            
            a = np.copy(generic_string)
            for j in subset:
                
                a[int(j)-1] = 1
                
            binary_list.append(a)
            
        return binary_list
    
    
    def __init__(self, n, task_num, cnf=0.1):
        
        
        # Numerical Parameters:
        # -------------------- #
        self.n = n
        self.num_of_tasks = task_num
        self.done = False
        self.cnf = cnf # communication noise factor of singleton value 
        # -------------------- #
        
        
        # Data Arrays:
        # ---------- #
        self.CS = [set() for i in range(1, self.num_of_tasks + 1)] # starting coalition structure 
        self.coalition_dict = {} # track the location of each agent in the coalition structure 
        self.single_dict = {}
        self.stability_dict = {}
        # ---------- #
        
        
        # Initialisation:
        # -------------- #
        for player in range(self.n): # choose a random task for each agent
            
            desired_coalition = random.randint(0, self.num_of_tasks-1) # pick a random coalition...
            self.CS[desired_coalition].add(f'{player +1 }') # ... and add the player to it...
            
            self.coalition_dict[f'Player {player + 1}'] = desired_coalition # ... while recording the index...
            self.single_dict[f'Player {player + 1}'] = random.random() # ...and its value ...
            self.stability_dict[f'Player {player + 1}'] = False # ... while setting all stability to false
        # -------------- #
        
        
    def step(self, actions): # expect 'actions' to be a dictionary list of actions for each player
        
        # Get new coalition structure:
        # --------------------------- #
        for player in range(self.n): # action will be index of task to join
            
            new_coalition = actions[player] # get new coalition index
            current_coalition = self.coalition_dict[f'Player {player + 1}'] # get old coalition index
            
           
            self.CS[current_coalition].remove(f'{player + 1}') # remove from old coalition...
            self.CS[new_coalition].add(f'{player + 1}') # add player to new coalition...


            self.coalition_dict[f'Player {player + 1}'] = new_coalition # ... and update coalition map

            
        next_state = self.get_observations_from_CS(self.CS, self.coalition_dict, self.n)
        # --------------------------- #
        
        # Communicate singleton values between coalition players: 
        # ------------------------------------------------------ #
        communicated_vals = {}
        for player in range(self.n):
            s_val = self.single_dict[f'Player {player + 1}']
            communicated_vals[f'Player {player + 1}'] = s_val + s_val*random.uniform(-self.cnf, self.cnf)
        # ------------------------------------------------------ #
        
        
        # Get payoffs for coalitions from char func:
        # --------------------------------------- #
        char_vals = []
        real_sum = []
        for coalition in self.CS: # get value for each coalition in coalition structure
            
            if len(coalition) > 0:
                
                full_value = self.characteristic_function(coalition, self.single_dict)
                char_vals.append(full_value)
                
                sum_value = self.get_sum(coalition, communicated_vals)
                real_sum.append(sum_value)
                
            else:
                
                char_vals.append(0)
                real_sum.append(0)
        # --------------------------------------- #
                
        rewards = []
        for player in range(self.n): # determine how this will be divided for players within the coalitions
            
            location = self.coalition_dict[f'Player {player + 1}'] # get location of player
            r_val = self.single_dict[f'Player {player + 1}'] # individual player real singleton value
            c_val = communicated_vals[f'Player {player + 1}'] # individual player communicated singleton value
            
            coalition_val = char_vals[location] # value of the coalition the player is in
            comm_sum = real_sum[location] # sum of singleton values of coalition
            
            award = (c_val/comm_sum) * coalition_val
            
            # check stability
            if award > r_val:
            
                reward = award  
                self.stability_dict[f'Player {player + 1}'] = True
                
            else:
                
                reward = 0
                
            rewards.append(reward)
        # --------------------------------------- #
        
        
        # Check termination and return next step info:
        # ------------------------------------------- #    
        info = [self.CS]
            
        # compile next_state, reward, done, info and return
        return next_state, rewards, self.done, info
        # ------------------------------------------- #
        
        
    def reset(self, n, task_num, cnf=0.1):
        
            
        
        # Numerical Parameters:
        # -------------------- #
        self.n = n
        self.num_of_tasks = task_num
        self.done = False
        self.cnf = cnf # communication noise factor of singleton value 
        # -------------------- #
        
        
        # Data Arrays:
        # ---------- #
        self.CS = [set() for i in range(1, self.num_of_tasks + 1)] # starting coalition structure 
        self.coalition_dict = {} # track the location of each agent in the coalition structure 
        self.single_dict = {}
        self.stability_dict = {}
        # ---------- #
        
        
        # Initialisation:
        # -------------- #
        for player in range(self.n): # choose a random task for each agent
            
            desired_coalition = random.randint(0, self.num_of_tasks-1) # pick a random coalition...
            self.CS[desired_coalition].add(f'{player +1 }') # ... and add the player to it...
            
            self.coalition_dict[f'Player {player + 1}'] = desired_coalition # ... while recording the index...
            self.single_dict[f'Player {player + 1}'] = random.random() # ...and its value ...
            self.stability_dict[f'Player {player + 1}'] = False # ... while setting all stability to false
        # -------------- #
        
        
    
    def render(self):
        pass




In [29]:
env = CoopEnv(4,2)

In [31]:
state = env.reset(4,2)

ent_vals = []

while env.done == False:
    
    actions = np.random.randint(0, env.num_of_tasks, env.n) # pick a random coalition for each player
    next_state, rewards, done, info = env.step(actions)
    #print(info[0])
    print(next_state)
    state = next_state
    

[array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([0., 0., 0., 1.])]
[array([1., 1., 0., 0.]), array([1., 1., 0., 0.]), array([0., 0., 1., 1.]), array([0., 0., 1., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 1., 0., 0.]), array([1., 1., 0., 0.]), array([0., 0., 1., 1.]), array([0., 0., 1., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 

[array([1., 1., 0., 1.]), array([1., 1., 0., 1.]), array([0., 0., 1., 0.]), array([1., 1., 0., 1.])]
[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.])]
[array([1., 1., 0., 0.]), array([1., 1., 0., 0.]), array([0., 0., 1., 1.]), array([0., 0., 1., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 1., 0., 1.]), array([1., 1., 0., 1.]), array([0., 0., 1., 0.]), array([1., 1., 0., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 

[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 1., 0.]), array([0., 1., 0., 1.]), array([1., 0., 1., 0.]), array([0., 1., 0., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([0., 0., 0., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 

[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.])]
[array([1., 1., 0., 0.]), array([1., 1., 0., 0.]), array([0., 0., 1., 1.]), array([0., 0., 1., 1.])]
[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.])]
[array([1., 1., 0., 1.]), array([1., 1., 0., 1.]), array([0., 0., 1., 0.]), array([1., 1., 0., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 1., 0., 0.]), array([1., 1., 0., 0.]), array([0., 0., 1., 1.]), array([0., 0., 1., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 

[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 1., 0., 0.]), array([1., 1., 0., 0.]), array([0., 0., 1., 1.]), array([0., 0., 1., 1.])]
[array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([0., 0., 0., 1.])]
[array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([0., 0., 0., 1.])]
[array([1., 1., 0., 1.]), array([1., 1., 0., 1.]), array([0., 0., 1., 0.]), array([1., 1., 0., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.])]
[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.])]
[array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([0., 0., 0., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 

...

[array([1., 0., 1., 0.]), array([0., 1., 0., 1.]), array([1., 0., 1., 0.]), array([0., 1., 0., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 1., 0., 1.]), array([1., 1., 0., 1.]), array([0., 0., 1., 0.]), array([1., 1., 0., 1.])]
[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 1., 0., 1.]), array([1., 1., 0., 1.]), array([0., 0., 1., 0.]), array([1., 1., 0., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 1., 0., 1.]), array([1., 1., 0., 1.]), array([0., 0., 1., 0.]), array([1., 1., 0., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 

[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 1., 1.]), array([0., 1., 0., 0.]), array([1., 0., 1., 1.]), array([1., 0., 1., 1.])]
[array([1., 1., 0., 0.]), array([1., 1., 0., 0.]), array([0., 0., 1., 1.]), array([0., 0., 1., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([0., 0., 0., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([0., 0., 

[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 1., 0.]), array([0., 1., 0., 1.]), array([1., 0., 1., 0.]), array([0., 1., 0., 1.])]
[array([1., 1., 0., 1.]), array([1., 1., 0., 1.]), array([0., 0., 1., 0.]), array([1., 1., 0., 1.])]
[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 0., 1., 0.]), array([0., 1., 0., 1.]), array([1., 0., 1., 0.]), array([0., 1., 0., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([0., 0., 

[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.])]
[array([1., 1., 0., 1.]), array([1., 1., 0., 1.]), array([0., 0., 1., 0.]), array([1., 1., 0., 1.])]
[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.])]
[array([1., 1., 0., 0.]), array([1., 1., 0., 0.]), array([0., 0., 1., 1.]), array([0., 0., 1., 1.])]
[array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 1., 0.]), array([1., 0., 0., 1.])]
[array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([1., 1., 1., 0.]), array([0., 0., 0., 1.])]
[array([1., 1., 0., 0.]), array([1., 1., 0., 0.]), array([0., 0., 1., 1.]), array([0., 0., 1., 1.])]
[array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1.])]
[array([1., 0., 1., 0.]), array([0., 1., 0., 1.]), array([1., 0., 1., 0.]), array([0., 1., 0., 1.])]
[array([1., 0., 0., 0.]), array([0., 1., 1., 1.]), array([0., 1., 1., 1.]), array([0., 1., 

KeyboardInterrupt: 